In [204]:
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets
import matplotlib.pyplot as plt
%matplotlib inline

In [205]:
df = pd.read_csv('building_inventory.csv',
                 na_values = {'Square Footage': 0,
                       'Year Acquired': 0,
                       'Year Constructed': 0})

In [ ]:
xl = df['Year Acquired'].values
yl = df['Square Footage'].values

x = df['cd']
y = df['An']
z = df['Square Footage']

xnames = x.unique()
ynames = y.unique()
znames = np.zeros([len(xnames),len(ynames)])

for i,xn in enumerate(xnames):
    for j,yn in enumerate(ynames):
        mask = (x == xn) & (y == yn)
        znames[i,j] = z[mask].sum()
znames_log = np.log10(znames)
znames_log[znames==0] = 0

In [226]:
# 2. Scales
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()

# 3. Axis
x_ax = bqplot.Axis(label='Year Constructed',scale = x_scl)
y_ax = bqplot.Axis(label='total square footage acquired', scale=y_scl,
                   orientation='vertical',side='left')

xl = df['Year Constructed'].values
yl = df['Square Footage'].values
# 4. Marks
i,j = 0,0
mask = (x.values == xnames[i] & (y.values == ynames[j]))
year = xl[mask]
sq = yl[mask]

#sq = sq[-pd.isnunll(year)]
#year = year[-pd.isnull(year)]

years = np.unique(year)
squ = [sq[year == years[i]].sum() for i in range(len(years))]

lines = bqplot.Lines(x = years, y = squ,
                    scales = {'x':x_scl, 'y':y_scl})

fig_lines = bqplot.Figure(marks = [lines], axes = [y_ax,x_ax])

fig_lines

Figure(axes=[Axis(label='total square footage acquired', orientation='vertical', scale=LinearScale(), side='le…

In [227]:
heatmap_matrix = pd.pivot_table(df,
                                index=['cd'],
                                values=['Square Footage'],
                                columns=['An'],
                                aggfunc=np.sum)

In [228]:
an = heatmap_matrix.columns.levels[1].to_list()

In [229]:
col_sc = bqplot.ColorScale(scheme='RdPu')
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

In [230]:
heatmap = bqplot.GridHeatMap(color = np.log10(heatmap_matrix.values),
                            row = heatmap_matrix.index,
                            column = an,
                            scales = {'color': col_sc,
                                     'row': y_sc,
                                     'column': x_sc})

In [231]:
fig = bqplot.Figure(marks = [heatmap], axes = [x_ax, y_ax, x_ax])
fig

Figure(axes=[Axis(label='Year Constructed', scale=LinearScale(), side='bottom'), Axis(label='total square foot…

In [242]:
def get_data_value(change):
    i,j = change['owner'].selected[0]
    mask = (x.values == xnames[i] & (y.values == ynames[j]))
    year = xl[mask]
    sq = yl[mask]
    
    years = np.unique(year)
    squ = [sq[year == years[b]].sum() for b in range(len(years))]
    
    lines.x = years
    lines.y = squ

heatmap.observe(get_data_value, 'selected')

fig.layout.max_width = '1000px'
fig.layout.max_height = '1000px'
fig.layout.min_width = '500px'
fig_lines.layout.min_width = '500px'
fig.layout.align_items

ipywidgets.HBox([fig, fig_lines])